<a href="https://colab.research.google.com/github/tiara070403/seminar/blob/main/Salinan_dari_seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install scikit-learn
!pip install Sastrawi
!pip install transformers
# Import Library
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from wordcloud import WordCloud
import nltk
nltk.download('punkt')
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from transformers import pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
df = pd.read_csv('/content/crawling.csv', index_col=0)

In [37]:
df = df[['created_at', 'full_text']]

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5349 entries, 1858070162362204643 to 1230765438897815552
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  5349 non-null   object
 1   full_text   5349 non-null   object
dtypes: object(2)
memory usage: 125.4+ KB


In [39]:
df['full_text'] = df['full_text'].replace('false', pd.NA)
df = df.dropna(subset=['full_text'])

In [40]:
# Menghapus Duplikat
initial_duplicates = df.duplicated().sum()
df = df.drop_duplicates(subset=['full_text'])
print(f"Jumlah duplikat awal: {initial_duplicates}")
print(f"Jumlah duplikat setelah drop: {df.duplicated().sum()}")

Jumlah duplikat awal: 4
Jumlah duplikat setelah drop: 0


In [41]:
# Menghapus baris dengan nilai null (setelah dropna subset)
print(f"Jumlah nilai null setelah dropna: \n{df.isnull().sum()}")

Jumlah nilai null setelah dropna: 
created_at    0
full_text     0
dtype: int64


In [42]:
# Cleaning
def clean_twitter_text(text):
  text = re.sub(r'@[A-Za-z0-9_]+', '', text) # Menghapus mention
  text = re.sub(r'#\w+', '', text)          # Menghapus hashtag
  text = re.sub(r'RT[\s]+', '', text)        # Menghapus RT
  text = re.sub(r'https?://\S+', '', text)   # Menghapus URL
  text = re.sub(r'[^A-Za-z0-9 ]', '', text) # Menghapus karakter non-alfabet
  text = re.sub(r'\s+', ' ', text).strip()   # Menghapus spasi berlebih
  return text
df['full_text'] = df['full_text'].apply(clean_twitter_text)
print(f"\nShape setelah Cleaning: {df.shape}")
print("DataFrame setelah Cleaning:")
print(df.head())


Shape setelah Cleaning: (5246, 2)
DataFrame setelah Cleaning:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643            lucu banget mahasiswa pencinta alam ini  
1846529554657239219  Bejat Mahasiswa Pencinta Alam di Jambi Genjot ...  
1467691301155979267  Ada Perkoempoelan Pentjinta Alam yang lahir th...  
1836014724523458593  PRESS RELEASE SOWAN UKM KE UKM UNIT PANDU LING...  
1835164951125917967  PRESS RELEASE SOWAN UKM KE UKM UNIT PANDU LING...  


In [48]:
# Normalisasi Data
norm = {
    'kekekegiatanananan': 'kegiatan', 'serang': 'serangan', 'paksa': 'memaksa', 'rekam': 'merekam', 'diri': 'didirikan',
    'terbit': 'menerbitkan', 'sehingtidak': 'sehingga', 'sebekamum': 'sebelum', 'univ': 'universitas', 'siar': 'siaran',
    'unjung': 'mengunjungi', 'lingkuntidaknan': 'lingkungan', 'kekekegiatananan': 'kegiatan', 'kamuring': 'di samping',
    'sekretariattariattariattariattariat': 'sekretariat', 'netidakra': 'netral', 'udah': 'sudah', 'obok': 'mengobok-obok',
    'ekamu': 'kamu', 'tungtidakngi': 'menunggangi', 'politik': 'politik', 'dih': 'dih', 'bjir': '', 'e': '', 'ngtidak': 'tidak',
    'berimpact': 'berdampak', 'mapatidakma': 'mapagama', 'universitas': 'universitas', 'tidakdjah': 'gadjah', 'laku': 'melakukan',
    'didik': 'pendidikan', 'gladimadiaa': 'gladimadya', 'kibar': 'mengibarkan', 'dentidakn': 'dengan', 'ad': 'ada', 'alamtidak': 'alam',
    'tidakib': 'takjub', 'bertidakbung': 'bergabung', 'daki': 'mendaki', 'bekerjaberbersama': 'bekerja sama', 'mentidakdakan': 'mengadakan',
    'sekakar': 'sekarang', 'laksana': 'laksanakan', 'melakamui': 'melalui', 'dukm': 'dulu', 'pancing': 'dipancing',
    'kekegiatanan': 'kegiatan', 'memmemmemmemmembukan': 'membuka', 'aberberberbersamaatis': 'berisiko', 'picu': 'memicu',
    'lakamu': 'wasalam', 'nyusuri': 'menyusuri', 'x': 'kali', 'dikamuar': 'di luar', 'ngingetin': 'mengingatkan', 'lat': 'saat',
    'alir': 'aliran', 'suntidaki': 'sungai', 'kec': 'kecamatan', 'kab': 'kabupaten', 'temu': 'ditemukan', 'olahratidak': 'olahraga',
    'bengkal': 'bengkalis', 'ampnbspmahasiswa': 'mahasiswa', 'sampai': 'menyampaikan', 'terlakamu': 'terlalu', 'kudu': 'harus',
    'hutan2': 'hutan-hutan', 'kalo': 'kalau', 'ortidaknisasi': 'organisasi', 'kaya': 'seperti', 'guhingtidakurian': 'guhingturian',
    'tentidakh': 'tengah', 'menkerenelar': 'menggelar', 'angkrintidakn': 'angkringan', 'ntidakliyan': 'naliyan',
    'kating': 'kakak tingkat', 'kekegiatananan': 'kegiatan', 'mentidakdakan': 'mengadakan', 'bakat': 'bakti',
    'memmemmemmemmembuka': 'membuka', 'tangtidakl': 'tanggal', 'hingtidak': 'hingga', 'kakkerenu': 'akhirnya',
    'kekegiatanan': 'kegiatan', 'ankerenota': 'anggota', 'pantau': 'memantau', 'lahatsit': 'lahat', 'msih': 'masih',
    'himpun': 'Himpunan', 'termemmemmemmemmembuka': 'terbuka', 'ucap': 'mengucapkan', 'ortidaknisasiortidaknisasi': 'organisasi',
    'memmemmemmemmembukan': 'membutuhkan', 'gusur': 'menggusur', 'serrifikat': 'sertifikat', 'mahasiwamahasiswa': 'mahasiswa',
    'belakantidakn': 'belakangan', 'mekakta': 'melihat', 'mahasiswamahasiswa': 'mahasiswa', 'blm': 'belum', 'suruh': 'disuruh',
    'apaaa': 'apa', 'kalo': 'kalau', 'tarik': 'tertarik', 'kaliann': 'kalian', 'mentidaktakan': 'menyampaikan', 'lingkuntidaknan': 'lingkungan',
    'sebekamum': 'sebelum', 'tidakisss': 'sih', 'pake': 'pakai', 'jelang': 'menjelang', 'kekegiatanan': 'kegiatan', 'rangkai': 'rangkaian',
    'kab': 'kabupaten', 'alamsosial': 'alam', 'laksana': 'melaksanakan', 'didik': 'pendidikan', 'kekekegiatananan': 'kegiatan',
    'ankerenota': 'anggota', 'biar': 'agar', 'dentidakn': 'dengan', 'tertidaktidakbung': 'tergabung', 'kena': 'menggunakan',
    'latih': 'latihan', 'ikut': 'diikuti', 'berbatidaki': 'berbagai', 'laku': 'melakukan', 'pemudapemudi': 'pemuda-pemudi',
    'masingmasing': 'masing-masing', 'bimbintidakn': 'bimbingan', 'menjatidak': 'menjadi', 'petidakng': 'petualang',
    'rombontidakn': 'rombongan', 'sebatidaki': 'sebagai', 'anakanak': 'anak-anak', 'bimbintidaknnya': 'bimbingannya',
    'masingmasing': 'masing-masing', 'penantidaknan': 'penanganan', 'jarintidakn': 'jaringan', 'kamunak': 'kamu',
    'dihingtidakkan': 'disampaikan', 'diklatsar': 'diklat', 'jemput': 'menjemput', 'sekamuruh': 'seluruh', 'tarik': 'tertarik',
    'lingkuntidaknan': 'lingkungan', 'tidakya': 'dia', 'pimpin': 'pemimpin', 'berbersamaberbersama': 'bersama', 'tidaknjar': 'ganjar',
    'termemmemmemmemmembuka': 'terbuka', 'nyari': 'mencari', 'kirakira': 'kira-kira', 'kliklinkdibawah': 'klik link di bawah',
    'menkerenelar': 'menggelar', 'rangkai': 'rangkaian', 'didik': 'pendidikan', 'ankerenota': 'anggota', 'angkat': 'angkatan',
    'sebekamum': 'sebelum', 'mentidakrungi': 'mengelilingi', 'rupa': 'berupa', 'rangkai': 'rangkaian', 'didik': 'pendidikan',
    'ankerenota': 'anggota', 'tp': 'tapi', 'bgt': 'banget', 'si': 'sih', 'blagi': 'lagi', 'marpala': 'mapala', 'nye': 'nya',
    'meningtidaklnya': 'meninggalnya', 'sampai': 'menyampaikan', 'kekekegiatananan': 'kegiatan', 'henti': 'dihentikan',
    'semenantia': 'sementara', 'susul': 'menyusul', 'bertidakbung': 'bergabung', 'laku': 'melakukan', 'daki': 'pendakian',
    'sebatidaki': 'sebagai', 'buat': 'membuat', 'perediaan': 'persediaan', 'kakum': 'kakimu', 'cari': 'mencari',
    'suntidaki': 'sungai', 'ankerenota': 'anggota', 'kesekretariattariattariattariattariattariatan': 'ke sekretariat',
    'apaa': 'apa', 'mahacitamahasiswa': 'mahacita', 'yawdahh': 'yaudahh', 'semantidakttt': 'semangat', 'latihannyaa': 'latihannya',
    'himpun': 'himpunan', 'raih': 'meraih', 'sebatidaki': 'sebagai', 'tidakbung': 'bergabung', 'kakimal': 'kakimu',
    'tidakk': 'tidak', 'beri': 'memberi', 'dukuntidakn': 'dukungan', 'tuju': 'tujuan', 'membangtidakkan': 'membangun',
    'laksana': 'melaksanakan', 'dentidakn': 'dengan', 'mahasiswa': 'mahasiswa', 'tahune': 'tahun','rangkai': 'rangkaian',
    'mapatidakma': 'mapagama', 'tidakdjah': 'gadjah', 'masa': 'mada', 'sambut': 'menyambut', 'sekamuruh': 'seluruh',
    'rangkai': 'rangkaian', 'awowkwowkok': '', 'nusanantia': 'nusantara', 'kenal': 'mengenal', 'olahratidak': 'olahraga',
    'inginmere': 'ingin', 'memperintidakti': 'memperingati', 'ke78': 'ke-78', 'lingkuntidaknan': 'lingkungan',
    'laksana': 'melaksanakan', 'laku': 'melakukan', 'natidakri': 'Ngarai', 'pukmh': 'puluh', 'titidak': 'tiga',
    'gladimadiaa': 'gladimadya', 'diri': 'terdiri', 'd': 'dan', 'olahratidak': 'olahraga', 'menkerenunakan': 'menggunakan',
    'mentidakrungi': 'mengelilingi', 'suntidaki': 'sungai', 'pinkerenir': 'pionir', 'suntidaki': 'sungai', 'mapatidakma': 'mapagama',
    'pacu': 'memacu', 'tangtidakl': 'tanggal', 'tidakdjah': 'gadjah', 'olahratidak': 'olahraga', 'bangsatt': '', 'sya': 'saya',
    'mw': 'mau', 'jd': 'jadi', 'ajh': 'aja', 'kakkerenu': 'kemarin', 'pukmhan': 'pemberi', 'menentidakh': 'menengah',
    'berberberbersamaa': 'bersama', 'ortidaknisasi': 'organisasi', 'dukung': 'mendukung', 'sama': 'bersama', 'tanam': 'menanam',
    'lingkuntidaknan': 'lingkungan', 'tidakib': 'takjub', 'kamuhur': 'luhur', 'perintidakti': 'memperingati',
    'lingkuntidaknan': 'lingkungan', 'tolak': 'menolak', 'kemas': 'kemasan', 'tuk': 'untuk', 'ni': 'ini',
    'lingkuntidaknan': 'lingkungan', 'kekamuartidak': 'komunitas', 'tidaknesha': 'ganesa', 'anjayy': 'anjay',
    'bett': 'banget', 'perintidakti': 'memperingati', 'bagi': 'membagikan', 'ananti': 'antar', 'beri': 'memberi',
    'sambut': 'sambutan', 'memmemmemmemmembuka': 'pembukaan', 'kehilantidakn': 'kehilangan',
    'soloyogyaraya': 'solo-yogyakarta', 'kumpul': 'berkumpul', 'panjat': 'memanjat', 'pergi': 'perginya',
    'lakamu': 'lalu', 'lpas': 'lepas', 'jmbar': 'jawa barat', 'akak': 'kakak', 'gw': 'gue', 'aza': 'aja',
    'puhingtidakik': 'praja', 'sampe': 'sampai', 'daki': 'mendaki', 'sebatidaki': 'sebagai', 'tidakrba': 'tarbiah',
    'didik': 'pendidikan', 'latih': 'latihan', 'diklatsar': 'diklat', 'kakkerenu': 'kemarin', 'kehilantidakn': 'kehilangan',
    'diri': 'anggota', 'ignya': 'ig-nya', 'bbrp': 'beberapa', 'rintis': 'merintis', 'ortidaknisasi': 'organisasi','tsb': 'tersebut',
    'mapasilvatidakma': 'mapasilva', 'daki': 'mendaki', 'rupa': 'berupa', 'ankerentidak': 'anggota', 'lingkuntidaknan': 'lingkungan',
    'kakkerenu': 'kemarin', 'ankerentidak': 'anggota', 'lingkuntidaknan': 'lingkungan', 'meningtidakl': 'meninggal',
    'ankerentidak': 'anggota', 'temu': 'pertemuan', 'tidakrut': 'garut', 'kekekegiatananan': 'kegiatan', 'setinkereni': 'setinggi',
    'himpun': 'himpunan', 'bertidaknti': 'berhenti', 'kekamuartidak': 'komunitas', 'kakkerenu': 'kemarin', 'ubiversitas': 'universitas',
    'termemmemmemmemmembuka': 'terbuka', 'kalo': 'kalau', 'ntidakk': 'tidak', 'batidakimana': 'bagaimana', 'kalo': 'kalau',
    'tertidaktidakbung': 'tergabung', 'didutidak': 'ditemukan', 'berberberbersamap': 'kelompok', 'dentidakn': 'dengan', 'tahune': 'the',
    'kamisjumat': 'kamis-jumat', 'kerjaberbersama': 'kerja sama', 'berberberbersamap': 'kelompok', 'dentidakn': 'dengan',
    'libat': 'melibatkan', 'bangun': 'membangun', 'ankerentidak': 'anggota', 'berkilokilo': 'berkilo-kilo',
    'mentidakdakan': 'mengadakan', 'lepas': 'pelepasan', 'menkerenelar': 'menggelar', 'maratahunon': 'marathon',
    'rupa': 'berupa', 'pakamu': 'untukmu', 'dongtidakla': 'donasi', 'lingkuntidaknan': 'lingkungan',
    'bekerjaberbersama': 'bekerja sama', 'dentidakn': 'dengan', 'memmemmemmemmemmembuka': 'membuka',
    'pengtidaklantidakn': 'penggalangan', 'tandatantidakni': 'tandatangani', 'rayaforsil': 'raya',
    'mememmemmemmemembuka': 'membuka', 'memmemmemmemmemmembantu': 'membantu', 'saudarasaudari': 'saudara-saudari',
    'pakamu': 'untukmu', 'semenantia': 'sementara', 'gelar': 'menggelar', 'siar': 'siaran', 'kawan2': 'kawan-kawan',
    'univ': 'universitas', 'pakamu': 'untukmu', 'cari': 'mencari', 'temu': 'ditemukan', 'wartidak': 'wartawan',
    'meningtidakl': 'meninggal', 'timbun': 'tertimbun', 'runtuh': 'runtuhan', 'bangun': 'bangunan', 'pasuk': 'masuk',
    'himpun': 'himpunan', 'pilih': 'terpilih', 'sebatidaki': 'sebagai', 'tetidakl': 'tegal',
    'kekekegiatananan': 'kegiatan', 'kawan2': 'kawan-kawan', 'terkamuar': 'terluar', 'ankerentidak': 'anggota',
    'diklatsar': 'diklat', 'memmemmemmemmemmembuka': 'membuka', 'berkekekegiatananan': 'berkegiatan',
    'meningtidakl': 'meninggal', 'pilih': 'terpilihnya', 'perintah': 'pemerintahan',
    'semotidak': 'semoga', 'jadi': 'menjadi', 'pimpin': 'pemimpin', 'ankerentidak': 'anggota',
    'berbatidaki': 'berbagai', 'tinkereni': 'tertinggi', 'dalamkebencanaan': 'dalam kebencanaan',
    'terkamuar': 'terluar', 'menyakamurkan': 'menyalurkan', 'hobbinya': 'hobinya',
    'silah': 'silakan', 'bertidakbung': 'bergabung', 'takkamukan': 'menaklukkan',
    'daki': 'mendaki', 'memmemmemmemmembukan': 'jaman', 'tinkereni': 'tinggi',
    'memmemmemmemmembukanlah': 'memanglah', 'memmemmemmemmembukanlah': 'bukanlah', 'catat': 'tercatat',
    'tertidaktidakbung': 'tergabung', 'menkerenelar': 'menggelar', 'maratahunon': 'marathon',
    'temu': 'menemukan', 'temu': 'ditemukan', 'butuh': 'membutuhkan', 'terima': 'menerima',
    'jadi': 'menjadi', 'tahune': 'the', 'butuh': 'membutuhkan', 'pengumumanpengumuman': 'pengumuman',
    'tidakbung': 'bergabung', 'tetangtidak': 'tetangga', 'wartidak': 'wartawan', 'netidakra': 'negara',
    'diri': 'terdiri', 'timbun': 'tertimbun', 'runtuh': 'runtuhan', 'jiwahartabenda': 'jiwa',
    'bekerjaberbersama': 'bekerja sama', 'memmemmemmemmemmembuka': 'membuka', 'jl': 'jalan',
    'telp021659775': 'telp 021-659775', 'sekretariattariattariattariattariattariat': 'sekretariat',
    'hpwa081385351171': 'hp/wa 081385351171', 'pertitidakan': 'pertigaan',
    'jltronojoyo': 'jl. trunojoyo', 'pengtidaklantidakn': 'penggalangan', 'tidakbuntidakn': 'gabungan',
    'tni122': 'tni 122', 'tidakbuntidakn': 'gabungan', 'diri': 'terdiri',
    'polrisabhara': 'polri-sabhara', 'tenatidak': 'tenaga', 'sama': 'bersama', 'tinkereni': 'tinggi',
    'tidakdjah': 'gadjah', 'sepukmh': 'sebelas', 'kakkerenu': 'kemarin', 'menkerenelar': 'menggelar',
    'panjang': 'sepanjang', 'alastuo': 'alas tuo', 'alastuo14com': 'alastuo14.com', 'dentidakn': 'dengan',
    'mentidakwal': 'menjaga', 'ortidaknisasi': 'organisasi', 'tahune': 'the', 'kendara': 'kendaraan',
    'tandatantidakni': 'tandatangani', 'sekakar': 'sekarang', 'ucap': 'mengucapkan',
    'terimakasihhhhhh': 'terima kasih', 'sekamuruh': 'seluruh', 'sekakar': 'sekarang', 'tarik': 'tertarik',
    'daerah2': 'daerah-daerah', 'teliti': 'meneliti', 'scr': 'secara', 'sebekamum': 'sebelum',
    'neliti': 'meneliti', 'ttg': 'tentang', 'idealiberberberbersamae': 'idealisme', 'wk': 'hahaha',
    'amp': 'dan', 'gtgt': '', 'sekretariattariattariattariattariatt': 'sekretariat', 'cari': 'mencari',
    'tahunr': 'tahun', 'kayak': 'kayak', 'aja': 'saja', 'dukm': 'dulu', 'rubah': 'merubah',
    'dentidakn': 'dengan', 'tualang': 'petualang', 'kekamuartidak': 'komunitas', 'ucap': 'mengucapkan',
    'dentidakn': 'dengan', 'kenal': 'mengenal', 'tidaknja': 'ganja', 'gerebek': 'digerebek',
    'adik2': 'adik-adik', 'tertidaktidakbung': 'tergabung', 'ortidaknisasi': 'organisasi',
    'timbukm': '', 'tertinkereni': 'tertinggi', 'perintidakti': 'memperingati',
    'sejawa': 'se-jawa', 'gelar': 'menggelar', 'tolak': 'menolak', 'teroriberberberbersamae': 'terorisme',
    'laku': 'dilakukan', 'tahune': 'the', 'selesai': 'menyelesaikan', 'lingkuntidaknan': 'lingkungan',
    'laku': 'melakukan', 'kekekegiatananan': 'kegiatan', 'lapantidakn': 'lapangan', 'selengtidkrakan': 'menyelenggarakan',
    'sekakar': 'acara', 'lingkuntidaknan': 'lingkungan', 'dentidakn': 'dengan', 'tumpah': 'tumpahan', 'kakyak': 'minyak',
    'tekamuk': 'tumpah', 'perintidakti': 'memperingati', 'bagibagi': 'membagikan',
    'bagibagi': 'membagikan', 'tidakkamuh': 'tidak', 'tidakmahasiswa': 'mahasiswa',
    'tidakk': 'tidak', 'mikak': 'kak', 'tau': 'tahu', 'hehedisimak': 'hehe disimak',
    'yahehe': 'ya hehe', 'memperintidakti': 'memperingati', 'pilih': 'terpilihnya',
    'sebatidaki': 'sebagai', 'memmemmemmemmembukanlah': 'bukanlah', 'ankerentidak': 'anggota',
    'ortidaknisasi': 'organisasi', 'istilah2': 'istilah-istilah', 'kekamuartidak': 'komunitas',
    'semotidak': 'semoga', 'dentidakn': 'dengan', 'mentidakdakan': 'mengadakan', 'rekrut': 'rekrutmen',
    'ankerentidak': 'anggota', 'hingtidak': 'hingga', 'perintidakti': 'memperingati', 'sekalimantan': 'se-kalimantan',
    'laku': 'melakukan', 'tolak': 'menolak', 'mititidaksi': 'mitigasi', 'bakar': 'kebakaran', 'jampe2kan': 'jampe-jampekan',
    'atidakr': 'agar', 'jadi': 'menjadi', 'org': 'orang', 'sembarantidakn': 'sembarangan', 'hrus': 'harus', 'y': 'ya',
    'bengkukm': 'bengku', 'upacara': 'upacara', 'pemmemmemmemmemmembukaan': 'pembukaan', 'perintidaktan': 'perayaan',
    'bengkukm': 'bengkulu', 'bpk': 'bapak', 'mentidakdakan': 'mengadakan', 'endahnrhesa': 'endah n rhesa', 'audensi': 'audiensi',
    'dentidakn': 'dengan', 'kait': 'terkait', 'lingkuntidaknan': 'lingkungan', 'melakamui': 'melalui', 'kenal': 'mengenal',
    'kex': 'ke-', 'dentidakn': 'dengan', 'cetidakh': 'cegah', 'laku': 'melakukan', 'hijau': 'penghijauan', 'siar': 'siaran',
    'jutidak': 'tidak', 'lingkung': 'lingkungan',
    'berberberbersama': 'bersama', 'press release': 'siaran pers', 'ukamu': 'ukm',
    'kekekekegiatanananan': 'kegiatan', 'jutidak': 'tidak', 'lingkung': 'lingkungan',
    'berberberbersama': 'bersama', 'press release': 'siaran pers', 'ukamu': 'ukm',
    'giat': 'kegiatan', 'luring': 'luring', 'sekre': 'sekretariat',
    'mapala': 'mahasiswa pencinta alam', 'kocak': 'lucu', 'enak': 'enak',
    'bilang': 'bilang', 'sowan': 'mengunjungi', 'buka': 'membuka',
    'sampai': 'hingga', 'pukul': 'jam', 'sd': 'sampai', 'kapan': 'kapan',
    'mau': 'ingin', 'join': 'bergabung', 'sama': 'bersama', 'bisa': 'dapat',
    'bantu': 'membantu', 'mikir2': 'mikir', 'perkoempoelan': 'perkumpulan',
    'genjot': 'menyerang', 'wkwkw': 'hahaha', 'th': 'tahun', 'bejat': 'rusak',
    'pentjinta': 'pencinta', 'siaaapp': 'siap', 'okaaay': 'oke', 'udh': 'sudah',
    'ga': 'tidak', 'gaskeun': 'ayo', 'wowww': 'wow', 'haaayyuukkk': 'ayo',
    'yg': 'yang', 'wkwk': '', 'min': 'kak', 'malem': 'malam', 'malem2': 'malam',
    'sm': 'sama', 'dy': 'dia', 'lg': 'lagi', 'skrg': 'sekarang', 'ddpn': 'didepan',
    'makasi': 'makasih', 'pertamaz': 'pertamax', 'jg': 'juga', 'donk': 'dong',
    'ikutann': 'ikutan', 'banyakk': 'banyak', 'twt': 'tweet', 'mantaap': 'mantap',
    'juarak': 'juara', 'daridulu': 'dari dulu', 'siapp': 'siap', 'gamau': 'tidak mau',
    'sll': 'selalu', 'qu': 'aku', 'krn': 'karena', 'irii': 'iri', 'muluu': 'terus',
    'mada': 'masa', 'jgn': 'jangan', 'muluuu': 'terus', 'ntar': 'nanti',
    'awtnya': 'awetnya', 'gg': 'keren', 'kerennn': 'keren', 'bisaa': 'bisa',
    'gaaa': 'tidak', 'nyampe': 'sampai', 'lu': 'kamu', 'ikhlaaasss': 'ikhlas',
    'gak': 'tidak', 'klo': 'kalo', 'kyk': 'seperti', 'sbg': 'sebagai',
    'giat': 'kegiatan', 'luring': 'luring', 'sekre': 'sekretariat',
    'mapala': 'mahasiswa pencinta alam', 'kocak': 'lucu', 'enak': 'enak',
    'bilang': 'bilang', 'sowan': 'mengunjungi', 'buka': 'membuka',
    'sampai': 'hingga', 'pukul': 'jam', 'sd': 'sampai', 'kapan': 'kapan',
    'mau': 'ingin', 'join': 'bergabung', 'sama': 'bersama', 'bisa': 'dapat',
    'bantu': 'membantu', 'mikir2': 'mikir', 'perkoempoelan': 'perkumpulan',
    'genjot': 'menyerang', 'wkwkw': 'hahaha', 'th': 'tahun', 'bejat': 'rusak',
    'pentjinta': 'pencinta', 'siaaapp': 'siap', 'okaaay': 'oke', 'udh': 'sudah',
    'ga': 'tidak', 'gaskeun': 'ayo', 'wowww': 'wow', 'haaayyuukkk': 'ayo',
    'yg': 'yang', 'wkwk': '', 'min': 'kak', 'malem': 'malam', 'malem2': 'malam',
    'sm': 'sama', 'dy': 'dia', 'lg': 'lagi', 'skrg': 'sekarang', 'ddpn': 'didepan',
    'makasi': 'makasih', 'pertamaz': 'pertamax', 'jg': 'juga', 'donk': 'dong',
    'ikutann': 'ikutan', 'banyakk': 'banyak', 'twt': 'tweet', 'mantaap': 'mantap',
    'juarak': 'juara', 'daridulu': 'dari dulu', 'siapp': 'siap', 'gamau': 'tidak mau',
    'sll': 'selalu', 'qu': 'aku', 'krn': 'karena', 'irii': 'iri', 'muluu': 'terus',
    'mada': 'masa', 'jgn': 'jangan', 'muluuu': 'terus', 'ntar': 'nanti',
    'awtnya': 'awetnya', 'gg': 'keren', 'kerennn': 'keren', 'bisaa': 'bisa',
    'gaaa': 'tidak', 'nyampe': 'sampai', 'lu': 'kamu', 'ikhlaaasss': 'ikhlas',
    'gak': 'tidak', 'klo': 'kalo', 'kyk': 'seperti', 'sbg': 'sebagai'
}
def normalisasi(str_text):
  for i in norm:
    str_text = str_text.replace(i, norm[i])
  return str_text
df['full_text'] = df['full_text'].apply(lambda x: normalisasi(x))
print("\nNormalisasi:")
print(df.head())
df


Normalisasi:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643    kamucu bangt mahasihhhhswa pncinta alam iiiiini  
1846529554657239219  bjat mahasihhhhswa pncinta alam danananaiiiini...  
1467691301155979267  adanaanaanaana prkompolan pntjinta alam yaaaaa...  
1836014724523458593  prss rlas mngmngmngunjungiii ukm k ukm uiiinit...  
1835164951125917967  prss rlas mngmngmngunjungiii ukm k ukm uiiinit...  


,created_at,full_text
conversation_id_str,,
1858070162362204643,Sun Nov 17 08:50:22 +0000 2024,kamucu bangt mahasihhhhswa pncinta alam iiiiini
1846529554657239219,Wed Oct 16 12:32:07 +0000 2024,bjat mahasihhhhswa pncinta alam danananaiiiini...
1467691301155979267,Wed Oct 09 13:58:44 +0000 2024,adanaanaanaana prkompolan pntjinta alam yaaaaa...
1836014724523458593,Tue Sep 17 12:09:56 +0000 2024,prss rlas mngmngmngunjungiii ukm k ukm uiiinit...
1835164951125917967,Sun Sep 15 03:53:14 +0000 2024,prss rlas mngmngmngunjungiii ukm k ukm uiiinit...
...,...,...
1230768955653967873,Fri Feb 21 08:19:20 +0000 2020,stiap orang danananaiiiini dananananuiiiinia i...
1230767649816100868,Fri Feb 21 08:14:08 +0000 2020,kapolri plstarian lingkuntidaknan mbbbberupaka...
1230765718263586818,Fri Feb 21 08:06:28 +0000 2020,plstarian lingkuntidaknan mbbbberupakan tankar...


No charts were generated by quickchart
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [44]:
# Case Folding
df['full_text'] = df['full_text'].str.lower()
print("\nDataFrame setelah Case Folding:")
print(df.head())


DataFrame setelah Case Folding:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643            lucu bangt mahasihswa pncinta alam iini  
1846529554657239219  bjat mahasihswa pncinta alam daini jambi gnjot...  
1467691301155979267  adana prkompolan pntjinta alam yaang lahir th ...  
1836014724523458593  press release sowan ukm ke ukm unit pandu ling...  
1835164951125917967  press release sowan ukm ke ukm unit pandu ling...  
